In [ ]:
!pip install negspacy
!pip install scispacy
!pip install stanza
!pip install spacy-stanza
!pip install spacy
import spacy
import stanza
import spacy_stanza
from negspacy.negation import Negex
from negspacy.termsets import termset
import pandas as pd
import numpy as np
import re as regular
from spacy.matcher import Matcher



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_ner_bc5cdr_md-0.5.1.tar.gz


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_ner_bc5cdr_md-0.5.1.tar.gz (120.2 MB)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
with open('/content/drive/MyDrive/AI_data/data1.txt','r') as f:
  e= f.read() 

FileNotFoundError: ignored

In [ ]:
data = e.split("-----")
a=data[0].split("\n\n")
records={}
i=0
j=0


for d in data:
    re = d.split("\n\n")

    records.update({i:re})
    i+=1
    
    if i>99:
        break
a=0
incomplete=[]
for i in range(100):
    if len(records[i]) ==3:
        
        a+=1
    else:
        incomplete.append(i)

Records={}
i=0
j=0


for d in data:
    re = d.split("\n\n")
    if i not in incomplete:
        j+=1
        Records.update({j:re})
    
        
    i+=1

with open("structureData.txt", 'w', encoding='utf8') as f:
    for key, value in Records.items():
        f.write('%s:%s\n' % (key, value))

In [ ]:
from negspacy.termsets import termset
ts = termset("en_clinical")
# customize the term list by adding more negation terms
ts.add_patterns({
            'preceding_negations': ['abstain from','other than','except for','except','with the exception of',
                                    'excluding','lack of','contraindication','contraindicated','interfere with',
                                   'prohibit','prohibits'],
            'following_negations':['negative','is allowed','impossible','exclusionary']
        })

In [ ]:
nlp_sd = spacy.load("en_ner_bc5cdr_md")
nlp_sd.add_pipe("negex", config={"ent_types":["CHEMICAL","DISEASE"]})
tt_nlp=spacy_stanza.load_pipeline('en', package='mimic', processors={'ner': 'i2b2'})
tt_nlp.add_pipe("negex", config={"ent_types":["TEST",'TREATMENT']})
nlp_age=spacy.load("en_core_web_sm")


INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: en (English):
| Processor | Package |
-----------------------
| tokenize  | mimic   |
| pos       | mimic   |
| lemma     | mimic   |
| depparse  | mimic   |
| ner       | i2b2    |

INFO:stanza:Use device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


In [ ]:
# for extracting gender
def gender(history,nlp_sd):
  sex = nlp_sd.get_pipe('attribute_ruler')
  sex.add([[{"TEXT":"male"}],[{"TEXT":"man"}],[{"TEXT":"boy"}],[{"TEXT":"him"}],[{"TEXT":"he"}],[{"TEXT":"his"}]],{"LEMMA":"Male"})
  sex.add([[{"TEXT":"female"}],[{"TEXT":"woman"}],[{"TEXT":"girl"}],[{"TEXT":"her"}],[{"TEXT":"she"}]],{"LEMMA":"Female"})
  sex_doc = nlp_sd(history)
  gender =["Male","Female"]
  for token in sex_doc:
      if str(token.lemma_) in gender:
          
          if token.lemma_ == "Male":
              
              return "M"
          else:
              return "F"
          break

In [ ]:
# for extracting age
def determineAge(history,nlp_age):
  
  doc_age=nlp_age(history)
  for ent in doc_age.ents:
    if ent.label_ == "DATE" and regular.search('years-old|years old|year-old|year old',ent.text):
      return ent.text


In [ ]:
#for extracting physical exam results
def extractPhyExam(examination, nlp_age):
  doc_e = nlp_age(examination)

  pattern = [{'LIKE_NUM': True}, 
           {"TEXT": {"IN": ["/min", "cm", "kg", "mmHg","m"]}},]
  matcher = Matcher(tt_nlp.vocab)
  matcher.add("TEST_RESULT", [pattern])

  matches = matcher(doc_e)  

  phy_exam = {}

  for match_id, start, end in matches:
          string_id = tt_nlp.vocab.strings[match_id]  # get string representation
          span = doc_e[start:end]  # the matched span       
          test_result = span.text
          if "kg" in test_result:
            phy_exam["Weight"] = test_result
          elif "mmHg" in span.text:
            phy_exam["Pressure"] = test_result
          elif "m" in test_result:
            phy_exam["Height"] = test_result

  return phy_exam


In [ ]:
def removePunc(text):
  print(text)
  punctuations = [',',"'",'"','?','!','_','(',')','[',']',':',';','{','}']
  for mark in punctuations:
    if text.__contains__(mark):
      return True
  return False

In [ ]:
patient_records = {}

bad_data = ['alcohol', 'treatment', 'examination', 'death']

punctuations = [',',"'",'"','?','!','_','(',')','[',']',':',';','{','}']

for i in range(1, len(Records)):
  symptoms =set()
  pre_medications =set()
  diagnosis = set()
  tests = set()
  treatments= set()
  age=determineAge(Records[i][0],nlp_age)
  sex=gender(Records[i][0],nlp_sd)
  #for extracting physical exam results
  phy_exam = extractPhyExam(Records[i][1], nlp_age)

  doc_s = nlp_sd(Records[i][0])
  doc_d = nlp_sd(Records[i][2])
  answer_doc_tt_nlp=tt_nlp(Records[i][2])
  


  for e in doc_s.ents:
    #symptom
      if str(e._.negex) == "False" and str(e.label_) == "DISEASE" and not removePunc(str(e.text)):
       symptoms.add(e.text)
       #past medications
      if str(e._.negex) == "False" and str(e.label_) == "CHEMICAL" and str(e.text).lower() not in bad_data and not removePunc(str(e.text)):
        pre_medications.add(e.text)
    #diagnosis
  for e in doc_d.ents:
      if str(e._.negex) == "False" and str(e.label_) == "DISEASE" and not removePunc(str(e.text)):
          diagnosis.add(e.text)

  
  for e in answer_doc_tt_nlp.ents:
      #test
      if str(e._.negex) == "False" and str(e.label_) == "TEST" and str(e.text).lower() not in bad_data and not removePunc(str(e.text)):
          tests.add(e.text)
      #treatment
      if str(e._.negex) == "False" and str(e.label_) == "TREATMENT" and str(e.text).lower() not in bad_data and not removePunc(str(e.text)):
          treatments.add(e.text)

  patient_records.update({i:{"Age":age,
                             "Sex":sex,
                              "Symptoms":list(symptoms),
                             "Pre_Medication":list(pre_medications),
                             "Physical Examination Results":phy_exam,
                             "Test":list(tests),
                             "Diagnosis":list(diagnosis),
                             "Treatment":list(treatments)}})

dizziness
gout
hypertrophy
ibuprofen
gout
dizziness
blackouts
loss of cardiac output usually associated withan arrhythmia
heart block
attacks
tachy-arrhythmias
heart block
asystole
heart block
dizziness
heart block
loss of consciousness splits into neurologicaland vascular
epilepsy
ischaemic attacks
vertebrobasilar insufficiency
pallor
arrhythmias
hypotension
faints
loss of consciousness
arrhythmias
heart block
cardiac output
blood flow return
chest X-ray
The ECG
the atrial rate
the ventricular rate
Local reduction
Differential diagnosisOn examination
the jugular venous pressure
a closed tricuspid valve
The treatment
a pacemaker
a permanent pacemaker
a dual-chamber system pacing
DDD
pacing
a ventricular pacing system
VVI
pacing the ventricle
ECG
chest pain
pain
chest pain
upper respiratory tract infection
sore throat
sneezing
cough
ill
migraine
myocardial infarction
cholesterol
infarct
thecholesterol
Cardiac pain
pain
chest pains
ischaemic heart disease
typicalof pain
myocardial infarc

In [ ]:
patient_records

{1: {'Age': '75-year-old',
  'Sex': 'M',
  'Symptoms': ['dizziness', 'gout', 'hypertrophy', 'blackouts'],
  'Pre_Medication': ['ibuprofen'],
  'Physical Examination Results': {'Pressure': '96/64 mmHg', 'Height': '3 cm'},
  'Test': ['the atrial rate',
   'the ventricular rate',
   'The ECG',
   'the jugular venous pressure',
   'chest X-ray',
   'Differential diagnosisOn examination',
   'ECG',
   'blood flow return',
   'cardiac output'],
  'Diagnosis': ['asystole',
   'pallor',
   'heart block',
   'tachy-arrhythmias',
   'epilepsy',
   'faints',
   'loss of consciousness',
   'arrhythmias',
   'dizziness',
   'vertebrobasilar insufficiency',
   'loss of consciousness splits into neurologicaland vascular',
   'attacks',
   'loss of cardiac output usually associated withan arrhythmia',
   'hypotension',
   'ischaemic attacks'],
  'Treatment': ['VVI',
   'pacing the ventricle',
   'a dual-chamber system pacing',
   'a closed tricuspid valve',
   'The treatment',
   'pacing',
   'a ventr

In [ ]:
import json
# with open('convert.txt', 'w') as convert_file:
#      convert_file.write(json.dumps(patient_records))
with open("finaldata.json", "w") as f:
  json.dump(patient_records,f)